# Clustering

Author: Tyler Reiser

In [7]:
import tsnmf

from sklearn.decomposition import TruncatedSVD, NMF
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime

from src.python.processor.data_processor import DataReader, BuildingProcessor
data_reader         = DataReader(start_date=datetime(2019,9,1),
                                 end_date=datetime(2019,12,16))
processor           = BuildingProcessor(data_reader)

In [8]:
from src.python.processor.data_processor import DataReader, BuildingProcessor
from datetime import datetime
data_reader     = DataReader(start_date=datetime(2021,2,10),end_date=datetime(2021,2,15))
data_processor  = BuildingProcessor(data_reader)

In [7]:
class DecompositionMethods:
    """
    A class to process SVD, TruncatedSVD, tsnmf, and NMF methods.
    """
    def __init__(self, n_components=15):
        self.n_components = n_components

    def apply_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        U, s, VT = np.linalg.svd(data_2d)
        S = np.zeros(data_2d.shape)
        S[:self.n_components, :self.n_components] = np.diag(s[:self.n_components])
        W = U @ S
        H = VT
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
        
    def apply_truncated_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = TruncatedSVD(n_components=self.n_components)
        model.fit(data_2d)
        W = model.transform(data_2d)
        H = model.components_
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
    
    def apply_nmf_sklearn(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = NMF(n_components=self.n_components, init='random', random_state=0)
        W = model.fit_transform(data_2d)
        H = model.components_
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

    def apply_tsnmf(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = tsnmf.smoothNMF(n_components=self.n_components)
        model.fit(data_2d)
        W = model.W
        H = model.H
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

class DecompositionVisualizer:

    def __init__(self, data_processor):
        self.decomp = DecompositionMethods()
        self.special_buildings = ['AERO', 'ATLS', 'C4C', 'LIBR', 'UMC', 'WLAW', 'REC', 'BCAPA', 'BCAPB']
        self.data = data_processor.process_all_buildings()

    def data_matrix(self, building):
        df  =   self.data[building]
        df['datetime'] = pd.to_datetime( df['datetime'])
        df.set_index('datetime', inplace=True)
        df['date'] = df.index.date
        df['time'] = df.index.time
        df['hours'] = df[ 'time'].apply(lambda t: (t.hour*3600 + t.minute*60 + t.second)/3600)
        df['week'] = df.index.isocalendar().week
        df['minutes'] = df.index.dayofweek * 24 * 60 + df['time'].apply(lambda t: t.hour * 60 + t.minute)
        df = df.groupby(['week', 'minutes'])['devicecount'].mean().unstack()

        return df

    def plot_and_save(self, data_matrix, building):
        print(f"Building: {building}")
        start_week = data_matrix.index.get_level_values('week').min()
        end_week = data_matrix.index.get_level_values('week').max()
        folder_name_date = f'Week {start_week} to {end_week}'
        building_title = building + f": Week {start_week} to {end_week}"
        
        fig = plt.figure(figsize=(14,20))
        gs = gridspec.GridSpec(3, 3, height_ratios=[1,0.5,0.5]) 

        plt.suptitle(building_title, fontsize=14, fontweight='bold')

        ax0 = plt.subplot(gs[0,:])
        ax1 = plt.subplot(gs[1,0])
        ax2 = plt.subplot(gs[2,0])
        ax3 = plt.subplot(gs[1,1])
        ax4 = plt.subplot(gs[2,1])
        ax5 = plt.subplot(gs[1,2])
        ax6 = plt.subplot(gs[2,2])

        for week, weekData in data_matrix.iterrows():
            ax0.plot(weekData.index / 1440, weekData, label=week, color='blue')

        ax0.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax0.xaxis.set_minor_locator(ticker.MultipleLocator(1/24))
        ax0.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'Day {int(x)}'))
        
        ax0.set_xlim([0, 7])
        ax0.set_ylim([0, data_matrix.max().max()])
        ax0.set_xlabel('Day of Week')
        ax0.set_ylabel('Device Count')
        ax0.set_title('ORIGINAL DATA - Device Count per day')
        ax0.legend()

        W_tsnmf, H_tsnmf, _ = self.decomp.apply_tsnmf(data_matrix)

        ax1.plot(W_tsnmf[:,:5])
        ax1.set_title('W - TSNMF (first 5 components)')
        ax2.plot(H_tsnmf[:5,:].T)
        ax2.set_title('H - TSNMF (first 5 components)')

        ax3.plot(W_tsnmf[:,5:10])
        ax3.set_title('W - TSNMF (next 5 components)')
        ax4.plot(H_tsnmf[5:10,:].T)
        ax4.set_title('H - TSNMF (next 5 components)')

        ax5.plot(W_tsnmf[:,10:])
        ax5.set_title('W - TSNMF (last 5 components)')
        ax6.plot(H_tsnmf[10:,:].T)
        ax6.set_title('H - TSNMF (last 5 components)')

        if building in self.special_buildings:
            folder_path = os.path.join(OUTPUT_PATH, 'special', folder_name_date, 'tsnmf')
        else:
            folder_path = os.path.join(OUTPUT_PATH, folder_name_date, 'tsnmf')
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = os.path.join(folder_path, f'{building}.png') 
        plt.savefig(file_path)
        plt.close()

    def apply_method(self, data_matrix):
        return self.decomp.apply_tsnmf(data_matrix)
    
    def plot_multiple(self):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix)
            self.plot_and_save(day_data_matrix, building)

    def plot_approximation_all_buildings(self):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix)
            approximation = np.dot(W, H)
            approx_df = pd.DataFrame(approximation, 
                                     index = day_data_matrix.index, 

                                     columns = day_data_matrix.columns)
            approx_series = approx_df.unstack()

    def plot_and_save_all_buildings(self):
        self.plot_multiple()
        

data_processor  = dp.DataProcessor(csv_directory='./data/input/WiFiData/', building_id='AERO')
data_visualizer = DecompositionVisualizer(data_processor)
data_visualizer.plot_multiple()

No file found with the name ./AERO_Extracted_Data_8-16-2019.csv.
No file found with the name /AERO_Extracted_Data_8-16-2019.csv.
No file found with the name d/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name a/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name t/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name a/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name /AERO_Extracted_Data_8-16-2019.csv.
No file found with the name i/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name n/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name p/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name u/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name t/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name /AERO_Extracted_Data_8-16-2019.csv.
No file found with the name W/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name i/AERO_Extracted_Data_8-16-2019.csv.
No file found with the name 

KeyError: 'Eduroam'

In [1]:
from src.python.utils import *
import src.python.DataProcessor as dp

from sklearn.decomposition import TruncatedSVD, NMF
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec

class DecompositionMethods:
    """
    A class to process SVD, TruncatedSVD, tsnmf, and NMF methods.
    """
    def __init__(self, n_components=4):
        self.n_components = n_components

    def apply_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        U, s, VT = np.linalg.svd(data_2d)
        S = np.zeros(data_2d.shape)
        S[:self.n_components, :self.n_components] = np.diag(s[:self.n_components])
        W = U @ S
        H = VT  
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
        
    def apply_truncated_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = TruncatedSVD(n_components=self.n_components)
        model.fit(data_2d)
        W = model.transform(data_2d)
        H = model.components_
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
    
    def apply_nmf_sklearn(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = NMF(n_components=self.n_components, init='random', random_state=0)
        W = model.fit_transform(data_2d)
        H = model.components_
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

    def apply_tsnmf(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values
        model = tsnmf.smoothNMF(n_components=self.n_components)
        model.fit(data_2d)
        W = model.W
        H = model.H
        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

class DecompositionVisualizer:

    def __init__(self, data_processor):
        self.decomp = DecompositionMethods()
        self.special_buildings = ['AERO', 'ATLS', 'C4C', 'LIBR', 'UMC', 'WLAW', 'REC', 'BCAPA', 'BCAPB']
        self.data = data_processor.process_all_buildings()
 
    def data_matrix(self, building):
        df  =   self.data[building]
        df[ 'datetime'  ]   =   pd.to_datetime( df[  'datetime'  ]  )
        df[ 'date'      ]   =   df[ 'datetime'  ].dt.date
        df[ 'time'      ]   =   df[ 'datetime'  ].dt.time
        df[ 'hours'     ]   =   df[ 'time'].apply(lambda t: (t.hour*3600 + t.minute*60 + t.second)/3600)
        df  =   df.groupby(['date', 'hours'])['devicecount'].mean().unstack()
        return df

    def plot_and_save(self, data_matrix, W, H, building, method):
        print(f"Building: {building}")
        
        start_date = data_matrix.index.min().strftime('%b-%d-%Y')
        end_date = data_matrix.index.max().strftime('%b-%d-%Y')
        folder_name_date = f'{start_date}_to_{end_date}'
        folder_name_interval = method
        building_title = building + f": {start_date} to {end_date}"
        
        fig = plt.figure(figsize=(20, 20))
        gs = gridspec.GridSpec(3, 3, height_ratios=[1, 0.5, 0.5]) 
    
        plt.suptitle(building_title, fontsize=14, fontweight='bold')

        ax0 = plt.subplot(gs[0, :])
        ax1 = plt.subplot(gs[1, 0])
        ax2 = plt.subplot(gs[2, 0])
        ax3 = plt.subplot(gs[1, 1])
        ax4 = plt.subplot(gs[2, 1])
        ax5 = plt.subplot(gs[1, 2])
        ax6 = plt.subplot(gs[2, 2])

        for index, dayData in data_matrix.iterrows():
            ax0.plot(dayData.index, dayData, label=index, color='blue')

        ax0.xaxis.set_major_locator(ticker.MultipleLocator(4))
        ax0.xaxis.set_minor_locator(ticker.MultipleLocator(1))
        ax0.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):02d}:{int(60*(x%1)):02d}'))
        
        ax0.set_xlim([0, 24])
        ax0.set_xlabel('Time of Day')
        ax0.set_ylabel('Device Count')
        ax0.set_title('ORIGINAL DATA - Device Count per day')
        ax0.legend()
            
        # Truncated SVD
        W_svd, H_svd, _ = self.decomp.apply_truncated_svd(data_matrix)
        ax1.plot(W_svd)
        ax1.set_title('W - Truncated SVD')
        ax2.plot(H_svd.T)
        ax2.set_title('H - Truncated SVD')

        # NMF
        W_nmf, H_nmf, _ = self.decomp.apply_nmf_sklearn(data_matrix)
        ax3.plot(W_nmf)
        ax3.set_title('W - NMF')
        ax4.plot(H_nmf.T)
        ax4.set_title('H - NMF')

        # TSNMF
        W_tsnmf, H_tsnmf, _ = self.decomp.apply_tsnmf(data_matrix)
        ax5.plot(W_tsnmf)
        ax5.set_title('W - TSNMF')
        ax6.plot(H_tsnmf.T)
        ax6.set_title('H - TSNMF')

        if building in self.special_buildings:
            folder_path = os.path.join(OUTPUT_PATH, 'special', folder_name_date, folder_name_interval)
        else:
            folder_path = os.path.join(OUTPUT_PATH, folder_name_date, folder_name_interval)
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = os.path.join(folder_path, f'{building}.png') 
        plt.savefig(file_path)
        plt.close()

        
    def apply_method(self, data_matrix, method):
        methods = {'tsnmf': self.decomp.apply_tsnmf,'nmf': self.decomp.apply_nmf_sklearn,'trunc-svd': self.decomp.apply_truncated_svd,}

        if method not in methods:
            raise ValueError(f"Invalid method: {method}")
        return methods[method](data_matrix)
    
    def plot_multiple(self, method):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix, method)
            self.plot_and_save(day_data_matrix, W, H, building, method)

    def plot_approximation_all_buildings(self, method):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix, method)
            approximation = np.dot(W, H)
            approx_df = pd.DataFrame(approximation, 
                                     index = day_data_matrix.index, 
                                     columns = day_data_matrix.columns)
            approx_series = approx_df.unstack()
            
    def plot_and_save_all_buildings(self):
        methods = ['tsnmf', 'nmf', 'trunc-svd']
        for method in methods:
            self.plot_multiple(method)
        


data_processor  = dp.DataProcessor(csv_directory = "CUT-CSV" )
data_visualizer = DecompositionVisualizer(data_processor)
data_visualizer.plot_multiple('nmf')

/Users/tylerreiser/Desktop/dev/Smart-Campus/src/python/DataProcessor.py:86: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_mat['datetime'] = pd.to_datetime(data_mat['datetime'], errors='coerce')
/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Building: ADEN


/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Stopping at iteration set in max_iter.


In [ ]:
    def data_matrix(self, building):
        df  =   self.data[building]
        df[ 'datetime'  ]   =   pd.to_datetime( df[  'datetime'  ]  )
        df[ 'date'      ]   =   df[ 'datetime'  ].dt.date
        df[ 'time'      ]   =   df[ 'datetime'  ].dt.time
        df[ 'hours'     ]   =   df[ 'time'].apply(lambda t: (t.hour*3600 + t.minute*60 + t.second)/3600)
        df  =   df.groupby(['date', 'hours'])['devicecount'].mean().unstack()

        return df

In [ ]:
---

# Matrix Decompositions
---

## Summary 


INITIALIZATION FOR NONNEGATIVE MATRIX FACTORIZATION: A COMPREHENSIVE REVIEW https://arxiv.org/pdf/2109.03874.pdf

NICA–NMF. The non-uniqueness (non-convexity)
property of NMF implies that the solution depends on the
initial factor matrices. To solve this problem we implement the
idea presented by Kitamura & Ono (2016) which suggests that
a good initialization is based on the factorization given by nonnegative ICA

The source above shows that NICA provideds best performance, so we want to show better performance and interpretability.

Nonnegative ICA: 
    - https://ieeexplore.ieee.org/document/1015161?denied=
    - https://github.com/Marius1311/Non-negative-ICA
    - 

Other useful references:
    - https://ieeexplore.ieee.org/document/759424
    - https://www.hindawi.com/journals/aav/2017/7132038/
    - https://onlinelibrary.wiley.com/doi/pdf/10.1107/S1600577523001674


Blind source separation. Blind source separation
(BSS) comprises all techniques that try to decouple a set of
source signals from a set of mixed signals with unknown (or
very little) information (Herault et al., 1985). Depending on
the assumptions on the data, different BSS techniques can
be used.


- The paper using PALM is available here: https://arxiv.org/abs/1910.14576

The class I made applies the time-series-nmf (TSNMF) library, a Python package specifically designed for time-series data that implements Non-negative Matrix Factorization (NMF). The TSNMF library supports Tikhonov regularization and sparse constraints and uses a Proximal Alternating Linearized Minimization (PALM) optimization scheme. The smoothNMF class in the TSNMF package implements NMF with a smoothness constraint, which is particularly helpful for time-series data, since adjacent time points are similar.

- The grant information for the Python package is available here: https://www.nsf.gov/awardsearch/showAward?AWD_ID=1849930&HistoricalAwards=false
- Information on the package are available at these two links: https://pypi.org/project/time-series-nmf/ and https://github.com/valentina-s/time-series-nmf/tree/v0.1.0.dev0

Within the smoothNMF class, `n_components` is a parameter that fixes the number of basis vectors the NMF algorithm will attempt to identify in the data. For example, in the current code, n_components is set to 5, indicating that the algorithm is seeking 5 basis vectors, also known as underlying patterns, in the device count dataset. The `fit` method in the smoothNMF class applies the NMF algorithm to the selected time interval. The `W` and `H` are attributes of the object created by the smoothNMF class--these represent the basis vectors and coefficients respectively. So, we are identifying underlying patterns in the data that are smooth as they move across time, and W and H matrices represent these patterns and their contributions to the original data over time. 

1. Since W represents the basis vectors (... or the patterns in the time-series data) with each line in the plot representing a basis vector (pattern), the vertical scale for W represents the magnitude of these patterns.

2. Since H represents the coefficients (... or the extent to which each pattern contributes to the original dat), each line in the plot represents the weight of a basis vector over time, and the vertical scale for H represents the magnitude of these contributions.

3. Recall that the time-series-nmf library applies NMF with a smoothness constraint. A smoothness constraint in time series analysis is a technique used to help identify trends and patterns in the data by reducing short-term fluctuations or noise. The idea is to make the time series smoother so that the underlying pattern is easier to see and understand. Equivallently, applying a smoothness constraint means that the algorithm will try to find basis vectors (W) and coefficients (H) that vary smoothly over time. So, the values of W and H at adjacent time points will be similar. This is particularly useful for time-series data, where the underlying patterns in the data are likely to change on a less-fast time scale. The patterns happen gradually rather than abruptly, and we observe that on a different time scale. The horizontal scale might seem to be all over the place due to this constraint, but the horizontal scales for both W and H represent time. The smoothness constraint improves the interpretability of the results, as the patterns identified by the NMF will be smoother and potentially easier to understand, and this will be helpful when we look at the "events happening within events" that we can see in the UMC pattern of life.

4. For the original data, the vertical scale is indeed device count, and the horizontal scale represents the specific timestamps the device count happens at.

Again, I am claiming that the time-series-nmf library applies this smoothness constraint when performing the NMF on the data. This is result in the horizontal scale of the W and H plots appearing to be "all over the place", as the smoothness constraint can cause the basis vectors and coefficients to change at a different rate over time. We just talked about something like this in my Computational Neuroscience class. I think, if this is true, then this would still work fine even if the original data has abrupt changes or is unevenly spaced. They are not unevenly spaced because of the interpolation but I did use a package to replace a lot of non-numeric entries with mean-valued entries. 

The Proximal Alternating Linearized Minimization (PALM) algorithm is a versatile method that can handle a broad class of nonconvex and nonsmooth minimization problems, making it a suitable choice for implementing the smoothness constraint in Non-negative Matrix Factorization (NMF) for time-series data. It is designed to solve a broad range of optimization problems with non-differentiable constraints. This makes it particularly well-suited for complex time-series data, which has high temporal variability (the change between time slices show significant differences even over short slices) and transient features (patterns that appear over time but disappear) that are usually very difficult to model with any accuracy.

- The article for the PALM algorithm: https://link.springer.com/article/10.1007/s10107-013-0701-9

In [3]:

from sklearn.decomposition import TruncatedSVD, NMF
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec


class DecompositionMethods:
    """
    A class to process SVD, TruncatedSVD, tsnmf, and NMF methods.
    """
    def __init__(self, n_components=4):
        self.n_components = n_components
        self.nica = nica.NonnegativeICA()

    def apply_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        U, s, VT = np.linalg.svd(data_2d)

        S = np.zeros(data_2d.shape)
        S[:self.n_components, :self.n_components] = np.diag(s[:self.n_components])

        W = U @ S
        H = VT

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')

        return W, H, objective
        
    def apply_truncated_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = TruncatedSVD(n_components=self.n_components)
        model.fit(data_2d)

        W = model.transform(data_2d)
        H = model.components_

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
    
    def apply_nmf_sklearn(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = NMF(n_components=self.n_components, init='random', random_state=0)
        W = model.fit_transform(data_2d)
        H = model.components_

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

    def apply_tsnmf(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = tsnmf.smoothNMF(n_components=self.n_components)
        model.fit(data_2d)

        W = model.W
        H = model.H

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

In [4]:
class DecompositionVisualizer:

    def __init__(self, data_processor):
        self.decomp = DecompositionMethods()
        self.special_buildings = ['AERO', 'ATLS', 'C4C', 'LIBR', 'UMC', 'WLAW', 'REC']
        self.data = data_processor.process_all_buildings()
 
    def data_matrix(self, building):
        df  =   self.data[building]
        df[ 'datetime'  ]   =   pd.to_datetime( df[  'datetime'  ]  )
        df[ 'date'      ]   =   df[ 'datetime'  ].dt.date
        df[ 'time'      ]   =   df[ 'datetime'  ].dt.time
        df[ 'hours'     ]   =   df[ 'time'].apply(lambda t: (t.hour*3600 + t.minute*60 + t.second)/3600)
        df  =   df.groupby(['date', 'hours'])['devicecount'].mean().unstack()

        return df

    def plot_and_save(self, data_matrix, W, H, building, method):
        
        print(f"Building: {building}")
        fig, axs = plt.subplots(3, 1, figsize=(15, 24))

        start_date = data_matrix.index.min().strftime( '%m-%Y' )
        end_date = data_matrix.index.max().strftime( '%m-%Y' )
        building_title = building + f": {start_date}_to_{end_date}"
        plt.suptitle(building_title, fontsize=14, fontweight='bold')

        for index, dayData in data_matrix.iterrows():
            axs[0].plot(dayData.index, dayData, label=index, color='blue)

        ax = axs[0]
        ax.xaxis.set_major_locator(ticker.MultipleLocator(4))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):02d}:{int(60*(x%1)):02d}'))
        
        axs[0].set_xlim([0, 24])
        axs[0].set_xlabel('Time of Day')
        axs[0].set_ylabel('Device Count')
        axs[0].set_title('ORIGINAL DATA - Device Count per day')
        axs[0].legend()
            
        axs[1].plot(W)
        axs[1].set_title('W')
        axs[2].plot(H.T)
        axs[2].set_title('H')

        start_date = data_matrix.index.min().strftime('%b-%d-%Y')
        end_date = data_matrix.index.max().strftime('%b-%d-%Y')
        folder_name_date = f'{start_date}_to_{end_date}'
        folder_name_interval = method

        if building in self.special_buildings:
            folder_path = os.path.join(OUTPUT_PATH, 'special', folder_name_date, folder_name_interval)
        else:
            folder_path = os.path.join(OUTPUT_PATH, folder_name_date, folder_name_interval)
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = os.path.join(folder_path, f'{building}.png') 
        plt.savefig(file_path)
        plt.close()

        
    def apply_method(self, data_matrix, method):
        methods = {
            'tsnmf': self.decomp.apply_tsnmf,
            'nmf': self.decomp.apply_nmf_sklearn,
            'trunc-svd': self.decomp.apply_truncated_svd,
            'svd': self.decomp.apply_svd
        }
        
        if method not in methods:
            raise ValueError(f"Invalid method: {method}")

        return methods[method](data_matrix)
    
    def plot_multiple(self, method):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix, method)
            self.plot_and_save(day_data_matrix, W, H, building, method)

    def plot_approximation_all_buildings(self, method):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix, method)
            approximation = np.dot(W, H)
            approx_df = pd.DataFrame(approximation, 
                                     index = day_data_matrix.index, 
                                     columns = day_data_matrix.columns)
            approx_series = approx_df.unstack()
            
    def plot_and_save_all_buildings(self):
        methods = ['tsnmf', 'svd', 'nmf', 'trunc-svd']
        for method in methods:
            self.plot_multiple(method)

SyntaxError: EOL while scanning string literal (2409228586.py, line 29)

In [ ]:
data_processor  = dp.DataProcessor(csv_directory = "CUT-CSV"
                                   )
data_visualizer = DecompositionVisualizer(data_processor)
data_visualizer.plot_multiple('nmf')

In [2]:
from src.python.utils import *
import src.python.DataProcessor as dp


from sklearn.decomposition import TruncatedSVD, NMF
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec


class DecompositionMethods:
    """
    A class to process SVD, TruncatedSVD, tsnmf, and NMF methods.
    """
    def __init__(self, n_components=4):
        self.n_components = n_components

    def apply_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        U, s, VT = np.linalg.svd(data_2d)

        S = np.zeros(data_2d.shape)
        S[:self.n_components, :self.n_components] = np.diag(s[:self.n_components])

        W = U @ S
        H = VT

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')

        return W, H, objective
        
    def apply_truncated_svd(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = TruncatedSVD(n_components=self.n_components)
        model.fit(data_2d)

        W = model.transform(data_2d)
        H = model.components_

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
    
    def apply_nmf_sklearn(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = NMF(n_components=self.n_components, init='random', random_state=0)
        W = model.fit_transform(data_2d)
        H = model.components_

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective

    def apply_tsnmf(self, data):
        data = data.fillna(data.mean())
        data_2d = data.values

        model = tsnmf.smoothNMF(n_components=self.n_components)
        model.fit(data_2d)

        W = model.W
        H = model.H

        approximation = np.dot(W, H)
        objective = np.linalg.norm(data_2d - approximation, 'fro')
        return W, H, objective
    
    
class DecompositionVisualizer:
    """
    A class to visualize SVD, TruncatedSVD, tsnmf, and NMF methods.
    """
    def __init__(self, data_processor):
        self.decomp = DecompositionMethods()
        self.special_buildings = ['AERO', 'ATLS', 'C4C', 'LIBR', 'UMC', 'WLAW', 'REC', 'BCAPA', 'BCAPB']
        self.data = data_processor.process_all_buildings()
 
    def data_matrix(self, building):
        df  =   self.data[building]
        df[ 'datetime'  ]   =   pd.to_datetime( df[  'datetime'  ]  )
        df[ 'date'      ]   =   df[ 'datetime'  ].dt.date
        df[ 'time'      ]   =   df[ 'datetime'  ].dt.time
        df[ 'hours'     ]   =   df[ 'time'].apply(lambda t: (t.hour*3600 + t.minute*60 + t.second)/3600)
        df  =   df.groupby(['date', 'hours'])['devicecount'].mean().unstack()

        return df
    
    def split_df_into_periods(df, num_periods):
        num_days        = df['date'].nunique()
        size, remainder = divmod(num_days, num_periods)
        sizes           = [size+1] * remainder + [size] * (num_periods - remainder)
        unique_dates    = df['date'].unique()
        unique_dates.sort()
        split_dates     = np.array_split(unique_dates, sizes)
        dataframes      = [df[df['date'].isin(dates)] for dates in split_dates]

        return dataframes

    def plot_and_save(self, data_matrix, W, H, building, method):
            print(f"Building: {building}")
            
            start_date = data_matrix.index.min().strftime('%b-%d-%Y')
            end_date = data_matrix.index.max().strftime('%b-%d-%Y')
            folder_name_date = f'{start_date}_to_{end_date}'
            folder_name_interval = method
            building_title = building + f": {start_date} to {end_date}"
            
            fig = plt.figure(figsize=(20, 10))
            gs = gridspec.GridSpec(2, 2, height_ratios=[1, 0.5]) 

            plt.suptitle(building_title, fontsize=14, fontweight='bold')

            ax0 = plt.subplot(gs[0, :])
            ax1 = plt.subplot(gs[1, 0])
            ax2 = plt.subplot(gs[1, 1])

            for index, dayData in data_matrix.iterrows():
                ax0.plot(dayData.index, dayData, label=index, color='blue')

            ax0.xaxis.set_major_locator(ticker.MultipleLocator(4))
            ax0.xaxis.set_minor_locator(ticker.MultipleLocator(1))
            ax0.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):02d}:{int(60*(x%1)):02d}'))
            
            ax0.set_xlim([0, 24])
            ax0.set_xlabel('Time of Day')
            ax0.set_ylabel('Device Count')
            ax0.set_title('ORIGINAL DATA - Device Count per day')
            ax0.legend()
                
            # Truncated SVD
            W_svd, H_svd, _ = self.decomp.apply_truncated_svd(data_matrix)
            ax1.plot(W_svd)
            ax1.set_title('W - Truncated SVD')
            ax2.plot(H_svd.T)
            ax2.set_title('H - Truncated SVD')
            
            plt.savefig(f"{building}_{method}_tsvd.png")

            # NMF
            plt.figure(figsize=(20, 5)) # Two subplots only (W and H) so no need to create grid
            ax1 = plt.subplot(1, 2, 1)
            ax2 = plt.subplot(1, 2, 2)

            W_nmf, H_nmf, _ = self.decomp.apply_nmf_sklearn(data_matrix)
            ax1.plot(W_nmf)
            ax1.set_title('W - NMF')
            ax2.plot(H_nmf.T)
            ax2.set_title('H - NMF')
            
            plt.savefig(f"{building}_{method}_nmf.png")

            # TSNMF
            plt.figure(figsize=(20, 5)) # Two subplots only (W and H) so no need to create grid
            ax1 = plt.subplot(1, 2, 1)
            ax2 = plt.subplot(1, 2, 2)
            W_tsnmf, H_tsnmf, _ = self.decomp.apply_tsnmf(data_matrix)
            ax1.plot(W_tsnmf)
            ax1.set_title('W - TSNMF')
            ax2.plot(H_tsnmf.T)
            ax2.set_title('H - TSNMF')

            plt.savefig(f"{building}_{method}_tsnmf.png")

        
    def apply_method(self, data_matrix, method):
        methods = {
            'tsnmf': self.decomp.apply_tsnmf,
            'nmf': self.decomp.apply_nmf_sklearn,
            'trunc-svd': self.decomp.apply_truncated_svd,
            'svd': self.decomp.apply_svd
        }
        
        if method not in methods:
            raise ValueError(f"Invalid method: {method}")

        W, H, _ = methods[method](data_matrix)
        
        return W, H, _  
    
    def plot_multiple(self, method):
        for building in self.data.keys():
            day_data_matrix = self.data_matrix(building)
            W, H, _ = self.apply_method(day_data_matrix, method)
            self.plot_and_save(day_data_matrix, W, H, building, method)
            
    def plot_and_save_all_buildings(self):
        methods = ['tsnmf', 'nmf', 'trunc-svd']
        for method in methods:
            self.plot_multiple(method)
        

data_processor  = dp.DataProcessor(csv_directory   =   './data/input/CUT-CSV/'
                                   )
data_visualizer = DecompositionVisualizer(data_processor)
data_visualizer.plot_multiple('nmf')

/Users/tylerreiser/Desktop/dev/Smart-Campus/src/python/DataProcessor.py:86: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_mat['datetime'] = pd.to_datetime(data_mat['datetime'], errors='coerce')
/Users/tylerreiser/Desktop/dev/Smart-Campus/src/python/DataProcessor.py:86: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_mat['datetime'] = pd.to_datetime(data_mat['datetime'], errors='coerce')
/Users/tylerreiser/Desktop/dev/Smart-Campus/src/python/DataProcessor.py:86: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_mat['datetime'] = pd.to_datetime(data_mat['datetime'], 

Building: ATLS


/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Stopping at iteration set in max_iter.
Building: UMC


/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


...

/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Stopping at iteration set in max_iter.


/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Building: ITLL


/Users/tylerreiser/Desktop/dev/Smart-Campus/env/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import os


from mpl_toolkits.mplot3d import Axes3D

# Plot all document spaces in 3D
def plot3D(var):
    n      = len(data)
    x      = np.arange(0, n)
    labels = ls.labels
    fig    = plt.figure(figsize=(10,10))
    ax     = fig.add_subplot(projection="3d")

    ax.set_xticks(xticks)
    ax.set_xticklabels(labels)
    
    ax.set_xlabel('Dimension', size=10)
    ax.set_ylabel('Document' , size=10)
    ax.set_zlabel('Magnitude', size=10)
    
    ax.view_init(elev=15., azim=-125)
     
    for i in range(var):
        ax.plot(x, data.T[i], i, 
                zdir      = 'y',
                linewidth = 0.6 )

plot3D(2000)

Noteworthy mentions:
 
1. Regularized Lotka-Volterra Dynamical System as Continuous Proximal-Like Method in Optimization
    - https://link.springer.com/article/10.1023/B:JOTA.0000037603.51578.45

2. A Proximal Minimization Algorithm for Structured Nonconvex and Nonsmooth Problems
    - https://epubs.siam.org/doi/abs/10.1137/18M1190689?journalCode=sjope8

3. On gradients of functions definable in o-minimal structures
    - https://eudml.org/doc/75302

4. A Decision Method for Elementary Algebra and Geometry:
    - https://www.rand.org/content/dam/rand/pubs/reports/2008/R109.pdf